### What is needed here

- Refresh the copy of GLODAP we have
- DONE: (temporary) Download the source data (five files)
- Create a wget step? Or subset the data into the repo... there are natively 33 levels
- DONE: Add cmocean as a dependency in binder/env.yml
- For Jupyterlab: pip install ipympl (?)
- Possibly also need what's below but the third line fails, argh


```
pip install nodejs
pip install npm
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install jupyter-matplotlib
```


#### From this point on the content is under revision June 2022

# Global Ocean Data Analysis Project


The objective for this notebook is to inspect layers of the global ocean by means of the GLODAP
dataset. 


The [GLODAP](https://en.wikipedia.org/wiki/Global_Ocean_Data_Analysis_Project) 
global ocean dataset compiles several decades of observation. It is a geological instant snapshot of
the state of the entire ocean, albeit downsampled a bit.


Moving the sliders for each chart selects for depth, as shown at the bottom left.

In [4]:
import os, sys, time, glob, warnings, cmocean

# from IPython.display import clear_output             # use inside loop with clear_output(wait = True) followed by print(i)
warnings.filterwarnings('ignore')

this_dir = os.getcwd()
data_dir = '/data/'                    # fix this

from matplotlib import pyplot as plt
from matplotlib import colors as mplcolors
import numpy as np, pandas as pd, xarray as xr
from ipywidgets import *
from traitlets import dlink

print('\nJupyter Notebook running Python {}'.format(sys.version_info[0]))

%matplotlib notebook


Jupyter Notebook running Python 3


In [5]:
%matplotlib widget

In [6]:
temperatureFnm = '../../data/glodap/temperature.nc'
salinityFnm    = '../../data/glodap/salinity.nc'
oxygenFnm      = '../../data/glodap/oxygen.nc'
NO3Fnm         = '../../data/glodap/no3.nc'
TCO2Fnm        = '../../data/glodap/tco2.nc'

dsSal  = xr.open_dataset(salinityFnm)
dsTemp = xr.open_dataset(temperatureFnm)
dsO2   = xr.open_dataset(oxygenFnm)
dsNO3  = xr.open_dataset(NO3Fnm)
dsTCO2 = xr.open_dataset(TCO2Fnm)

plt.rcParams.update({'font.size': 16})

def glodap_pO2(depth_index):
    dsO2['oxygen'].sel(depth_surface = depth_index).plot(figsize=(10.4, 6),cmap=cmocean.cm.oxy,vmin=150, vmax=350)
    plt.ylabel(None); plt.xlabel(None)
    if depth_index == 0: msg = 'This is for surface water'
    else:                msg = 'This is for water at ' + str(int(dsO2['Depth'].values[depth_index])) + ' meters depth'
    plt.text(25, -87, msg); plt.text(28, 50, 'oxygen dissolved in'); plt.text(28, 42, '     ocean water   ')

def glodap_pSal(depth_index):
    dsSal['salinity'].sel(depth_surface = depth_index).plot(figsize=(10.4, 6),cmap=cmocean.cm.haline,vmin=33, vmax=36)
    plt.ylabel(None); plt.xlabel(None)
    if depth_index == 0:msg = 'This is for surface water'
    else: msg = 'This is for water at ' + str(int(dsSal['Depth'].values[depth_index])) + ' meters depth'
    plt.text(25, -87, msg); plt.text(47, 50, 'salinity of'); plt.text(47, 42, 'ocean water')

def glodap_pTemp(depth_index):
    dsTemp['temperature'].sel(depth_surface = depth_index).plot(figsize=(10.4, 6),cmap=cmocean.cm.thermal,vmin=2., vmax=12.)
    plt.ylabel(None); plt.xlabel(None)
    if depth_index == 0: msg = 'This is for surface water'
    else: msg = 'This is for water at ' + str(int(dsTemp['Depth'].values[depth_index])) + ' meters depth'
    plt.text(25, -87, msg); plt.text(47, 50, 'temperature of'); plt.text(47, 42, 'ocean water')
    
def glodap_pNO3(depth_index):
    dsNO3['NO3'].sel(depth_surface = depth_index).plot(figsize=(10.4, 6),cmap=cmocean.cm.algae,vmin=2., vmax=40.)
    plt.ylabel(None); plt.xlabel(None)
    if depth_index == 0: msg = 'This is for surface water'
    else: msg = 'This is for water at ' + str(int(dsNO3['Depth'].values[depth_index])) + ' meters depth'
    plt.text(25, -87, msg); plt.text(47, 50, 'nitrate in'); plt.text(47, 42, 'ocean water')

def glodap_pTCO2(depth_index):
    dsTCO2['TCO2'].sel(depth_surface = depth_index).plot(figsize=(10.4, 6), cmap=cmocean.cm.delta, vmin=2000., vmax=2500.)
    plt.ylabel(None); plt.xlabel(None)
    if depth_index == 0: msg = 'This is for surface water'
    else: msg = 'This is for water at ' + str(int(dsTCO2['Depth'].values[depth_index])) + ' meters depth'
    plt.text(25, -87, msg); plt.text(47, 50, 'carbon dioxide in'); plt.text(47, 42, 'ocean water')

interact(glodap_pO2,   depth_index=widgets.IntSlider(min=0, max=32, step=1, value=0, continuous_update=False, description='depth'))
interact(glodap_pSal,  depth_index=widgets.IntSlider(min=0, max=32, step=1, value=0, continuous_update=False, description='depth'))
interact(glodap_pTemp, depth_index=widgets.IntSlider(min=0, max=32, step=1, value=0, continuous_update=False, description='depth'))
interact(glodap_pNO3,  depth_index=widgets.IntSlider(min=0, max=32, step=1, value=0, continuous_update=False, description='depth'))
interact(glodap_pTCO2, depth_index=widgets.IntSlider(min=0, max=32, step=1, value=0, continuous_update=False, description='depth'))

interactive(children=(IntSlider(value=0, continuous_update=False, description='depth', max=32), Output()), _do…

interactive(children=(IntSlider(value=0, continuous_update=False, description='depth', max=32), Output()), _do…

interactive(children=(IntSlider(value=0, continuous_update=False, description='depth', max=32), Output()), _do…

interactive(children=(IntSlider(value=0, continuous_update=False, description='depth', max=32), Output()), _do…

interactive(children=(IntSlider(value=0, continuous_update=False, description='depth', max=32), Output()), _do…

<function __main__.glodap_pTCO2(depth_index)>

## Set the slider above to 28

Compare with the mid-ocean ridges shown here.


<BR>
<img src="./../Images/charts/lithosphere_age.png" style="float: left;" alt="drawing" width="800"/>
<div style="clear: left"><BR>